## GIS 5572 Lab 3
I had to rerun part of Lab 2 in order to do this lab. Many thanks to Sebastian for letting me look at his code to get this part done.

In [25]:

import requests
import os
import datetime
import json



### Connect Postgres database
How to: https://pro.arcgis.com/en/pro-app/latest/help/data/databases/connect-postgresql.htm
Use Connections in toolbar
Add Database connection

In [ ]:
workspace = r"Z:\Helpdesk\Shana\GIS 5572\5572Lab3"

weather_data = 'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2024&month=2'

In [9]:
response = requests.get(weather_data)
file_path = os.path.join(workspace, 'weather_2.geojson')
with open(file_path, 'wb') as file:
    file.write(response.content)

In [10]:
out_features = 'weather2'
arcpy.conversion.JSONToFeatures(file_path, out_features, geometry_type="POINT")

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\5572Lab3.gdb\\weather2'>

In [20]:
arcpy.env.workspace = r"Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde"

weather_data = 'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2024&month=2'
out_features = 'weather2_1'
arcpy.conversion.JSONToFeatures(file_path, out_features, geometry_type="POINT")

<Result 'Z:\\\\Helpdesk\\\\Shana\\\\GIS 5572\\\\5572Lab3\\\\35.224.112.3.sde\\labgis5572.postgres.weather2_1'>

In [21]:

bad_data_ids = []

# Check the data points
with arcpy.da.SearchCursor(out_features, ["OBJECTID", "date", "min_tmpf", "max_tmpf"]) as cursor:
    for row in cursor:
        obj_id = row[0]
        date_col = row[1]
        min_temp = row[2]
        max_temp = row[3]
        
        date = datetime.datetime.strptime(date_col, '%Y-%m-%d')
        is_february = date.month == 2
        if min_temp is None:
            is_min_valid = False
        else: 
            is_min_valid = -30 <= min_temp <= 70
        if max_temp is None:
            is_max_valid = False
        else: 
            is_max_valid = -30 <= max_temp <= 70

        # Check if the date is within February, max_tmpf is betwen -30 and 70, and min_tmpf is between -30 and 70 
        if not is_february:
            print(f"Incorrect date, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        elif not is_min_valid:
            print(f"Incorrect min_tmpf, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        elif not is_max_valid:
            print(f"Incorrect max_tmpf, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        
print(bad_data_ids)   
     

Incorrect min_tmpf, ObjectID: 356
Incorrect min_tmpf, ObjectID: 519
Incorrect min_tmpf, ObjectID: 2779
Incorrect min_tmpf, ObjectID: 2942
Incorrect max_tmpf, ObjectID: 4076
Incorrect max_tmpf, ObjectID: 4084
Incorrect max_tmpf, ObjectID: 4171
Incorrect max_tmpf, ObjectID: 4210
[356, 519, 2779, 2942, 4076, 4084, 4171, 4210]


In [23]:
out_features = 'weather2'
where_clause = "OBJECTID IN ({})".format(", ".join(map(str, bad_data_ids)))
print(where_clause)
arcpy.SelectLayerByAttribute_management(out_features, "NEW_SELECTION", where_clause)
arcpy.DeleteFeatures_management(out_features)
     

OBJECTID IN (356, 519, 2779, 2942, 4076, 4084, 4171, 4210)


<Result 'weather2'>

In [24]:
arcpy.env.workspace = "Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde"
spatial_ref = arcpy.SpatialReference(4326)
out_name = "feb_weather_1"
geometry_type = "POINT"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, out_name, geometry_type, spatial_reference=spatial_ref)

     

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\feb_weather_1'>

In [34]:
# Path to your feature layer
feature_layer_path = r'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\labgis5572.postgres.weather2_1'
# Get a list of field names
field_names = [field.name for field in arcpy.ListFields(feature_layer_path)]

# Print the list of field names
print(field_names)

['objectid', 'station', 'date', 'max_tmpf', 'min_tmpf', 'precip', 'max_gust', 'snow', 'snowd', 'min_rh', 'max_rh', 'max_dwpf', 'min_dwpf', 'min_feel', 'avg_feel', 'max_feel', 'max_drct', 'precip_est', 'tmpf_est', 'max_gust_localts', 'temp_hour', 'avg_sknt', 'vector_avg_drct', 'min_rstage', 'max_rstage', 'id', 'name', 'shape']


In [35]:
bad_data = []

# Check the data points
with arcpy.da.SearchCursor(out_features, ["OBJECTID", "date", "min_tmpf", "max_tmpf"]) as cursor:
    for row in cursor:
        obj_id = row[0]
        date_col = row[1]
        min_temp = row[2]
        max_temp = row[3]
        
        date = datetime.datetime.strptime(date_col, '%Y-%m-%d')
        is_february = date.month == 2
        if min_temp is None:
            is_min_valid = False
        else: 
            is_min_valid = -30 <= min_temp <= 70
        if max_temp is None:
            is_max_valid = False
        else: 
            is_max_valid = -30 <= max_temp <= 70

        # Check if the date is within February, max_tmpf is betwen -30 and 70, and min_tmpf is between -30 and 70 
        if not is_february:
            print(f"Incorrect date, ObjectID: {obj_id}")
            bad_data.append(obj_id)
        elif not is_min_valid:
            print(f"Incorrect min_tmpf, ObjectID: {obj_id}")
            bad_data.append(obj_id)
        elif not is_max_valid:
            print(f"Incorrect max_tmpf, ObjectID: {obj_id}")
            bad_data.append(obj_id)
        
print(bad_data)   
     


Incorrect min_tmpf, ObjectID: 356
Incorrect min_tmpf, ObjectID: 519
Incorrect min_tmpf, ObjectID: 2779
Incorrect min_tmpf, ObjectID: 2942
Incorrect max_tmpf, ObjectID: 4076
Incorrect max_tmpf, ObjectID: 4084
Incorrect max_tmpf, ObjectID: 4171
Incorrect max_tmpf, ObjectID: 4210
[356, 519, 2779, 2942, 4076, 4084, 4171, 4210]


In [ ]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="labgis5572.postgres.weather2_1",
    selection_type="NEW_SELECTION",
    where_clause="min_tmpf <= -30 Or max_tmpf > 70",
    invert_where_clause=None
)

In [37]:
arcpy.env.workspace = "Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde"
spatial_ref = arcpy.SpatialReference(4326)
out_name = "feb_weather_100"
geometry_type = "POINT"
arcpy.management.CreateFeatureclass(arcpy.env.workspace, out_name, geometry_type, spatial_reference=spatial_ref)

<Result 'Z:\\Helpdesk\\Shana\\GIS 5572\\5572Lab3\\35.224.112.3.sde\\feb_weather_100'>

## Problem:
Everytime I try to save it as a feature class after doing the QA, no features show up. I'm not sure if I'm doing the deleting wrong or what. I finally went to the gui, and there it's telling me that there are no editable features to delete. 

I realized the problem is saving to the sde database. It kills Pro!